In [10]:
import numpy as np
import pandas as pd
import wfdb
from matplotlib import pyplot as plt
from preprocess import *
from sampling import *
from numpy import fft as fft
from sklearn.utils import shuffle
%matplotlib inline

In [ ]:
records = wfdb.get_record_list('mitdb')
directory = 'data/'
sample_rate = 360

### Store signals, annotation positions and annotation symbols in dictionaries

In [3]:
# signal data

SignalData = {}

for record in records:    
    signal, field = wfdb.rdsamp(directory + record)
    for channel in [0,1]:
        if field['sig_name'][channel] == 'MLII': # only use data from MLII
            SignalData[record + ' ' + field['sig_name'][channel]] = signal[:, channel]

In [4]:
# annotation data

AnnPos, AnnSym = {}, {}

for record in records:
    annotation = wfdb.rdann(directory + record, 'atr')
    AnnPos[record] = annotation.sample
    AnnSym[record] = annotation.symbol

### Compute segmentation positions
Segmentation positions are the middle points between 2 neighboring beats. They serve as the starting positions of the following beats. For the first beat in a record, the starting position is set to be 0.

In [5]:
SegPos = segmentation(AnnPos) # Segpos is a dictionary that contains all the segmentation data.

### Remove noise from the signal

In [6]:
for key in SignalData.keys():
    
    SignalData[key] = denoise(SignalData[key])

### Draw samples from the data with a rolling window.

Each sample contains:

1. signals collected in a feature derivation window (FDW)
2. beat annotations in a forecast window (FW)
And there is a time gap between FDW and FW.
The goal is to use 1 to predict 2.

In [7]:
# some parameters
FDW_width, FW_width, gap_width = 25, 1, 0  # widths(in number of beats) of the feature derivation window, forecast window, and the gap in between.
delay = 1 # distance(in number of beats) between two consequential FDWs.

In [13]:
DataSet, LabelSet = rolling_window(SignalData, SegPos, AnnSym, FDW_width, FW_width, gap_width, delay)

In [14]:
print('Number of samples is %d' %(len(DataSet)))
print('Minimum FDW in the samples contains ' + str(min(len(value) for value in DataSet.values())) + ' data points')

Number of samples is 106902
Minimum FDW in the samples contains 2215 data points


In [15]:
print('The last column in each array in DataSet is the patient information')
DataSet[0]

The last column in each array in DataSet is the patient information


array([ -6.71225956e-02,  -6.75534839e-02,  -6.75534839e-02, ...,
         2.52767827e-02,   2.39828495e-02,   1.00000000e+02])

### Make the data to be the same lengths: <br>
    Method 1: just crop out the beginning part of the arrays that is longer than the minimum length.

In [16]:
length = 2001

In [17]:
# cropping
for key in DataSet.keys():
    DataSet[key] = DataSet[key][-length:]

### Choose only forecast windows that contains an N or L beat,  downsample N beat, and build X/y matrix as machine learning input/output

In [42]:
# return keys of N and L beats in the dataset
N_keys = [key for key, value in LabelSet.items() if value == ['N']]
Ab_keys = [key for key, value in LabelSet.items() if value[0] in [
    'L','R','B','A','a','J','S','V','r','F','e','j','n','E','P','f','Q']]
# count numbers of N and L beats in the forecast window
nN, nAb = len(N_keys), len(Ab_keys)
nN, nAb

(73942, 26385)

In [31]:
# build X and y  (note that the last column in X is patient information)
X = np.zeros((nAb * 2, length))
y = np.zeros(nAb * 2)

N_keys_reduced = shuffle(N_keys, n_samples = nAb, random_state = 0) # downsample N beat samples.

i = 0
for key in N_keys_reduced: 
    y[i] = 0
    X[i,:] = DataSet[key]
    i = i + 1
for key in Ab_keys:
    y[i] = 1
    X[i,:] = DataSet[key]
    i = i + 1
X, y = shuffle(X, y, random_state = 0)

In [32]:
Xf = np.zeros((X.shape[0], X.shape[1] + 1))
for i in range(X.shape[0]):
    xf = fft.rfft(X[i,:-1]) # remove the last number in each array that represent patient information.
    Xf[i, 0:(int((X.shape[1] + 1)/2))] = np.real(xf)
    Xf[i, (int((X.shape[1] + 1)/2)):] = np.imag(xf)
Xf = np.append(Xf, X[:,-1][:,None], axis = 1) # add patient information to the last column.

### Split training and testing dataset

In [33]:
X_split, y_split = split_dataset_by_patient(Xf, y, [0.8, 0.2])
X_train, X_test = X_split[0], X_split[1]
y_train, y_test = y_split[0], y_split[1]

### Pipeline

In [25]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [34]:
pipe_nnw = Pipeline([('scl', StandardScaler()), ('pca', PCA(n_components = 200)),
                     ('clf', MLPClassifier(hidden_layer_sizes = (200,), solver = 'adam', 
                                           random_state = 0, max_iter = 200))])

In [ ]:
X_partition, y_partition = split_dataset_by_patient(X_train, y_train, [1/3, 1/3, 1/3])

In [ ]:
train_score, valid_score = [0, 0, 0], [0, 0, 0]
for i in range(3):
    
    valid_part = i
    train_part = [part for part in range(3) if part != valid_part]
    X_train_part = np.concatenate((X_partition[train_part[0]], X_partition[train_part[1]]), axis = 0)[:, :-1]
    X_valid_part = X_partition[valid_part][:, :-1]
    y_train_part = np.concatenate((y_partition[train_part[0]], y_partition[train_part[1]]))
    y_valid_part = y_partition[valid_part]
    
    pipe_nnw.fit(X_train_part, y_train_part)
    train_score[i] = pipe_nnw.score(X_train_part, y_train_part)
    valid_score[i] = pipe_nnw.score(X_valid_part, y_valid_part)